# CFB Mode (Cipher Feedback)

CFB (Cipher Feedback) is a mode of operation that converts a block cipher (such as AES) into a stream cipher. It enables the encryption of data in segments (even bit-by-bit or byte-by-byte) without requiring padding, making it suitable for encrypting messages of arbitrary lengths.

---

## How It Works

### Encryption Process

1. **Initialization:**
   - **IV:** A random, unique Initialization Vector is generated.
   - **First Keystream Block:**  
     `O₁ = Eₖ(IV)`  
     Here, `Eₖ` denotes encryption with key **K**.

2. **Block-by-Block Encryption:**
   - For the first plaintext segment **P₁**:  
     `C₁ = P₁ ⊕ O₁`
   - For subsequent segments, the previous ciphertext is fed back:
     - Compute:  
       `O₂ = Eₖ(C₁)`  
       Then,  
       `C₂ = P₂ ⊕ O₂`
     - Continue similarly for further blocks:
       ```
       O₃ = Eₖ(C₂)
       C₃ = P₃ ⊕ O₃
       ...
       ```

### Decryption Process

Decryption follows the same process using the same key and IV:
- **For the first ciphertext segment:**  
  `P₁ = C₁ ⊕ Eₖ(IV)`
- **For subsequent segments:**  
  ```
  P₂ = C₂ ⊕ Eₖ(C₁)
  P₃ = C₃ ⊕ Eₖ(C₂)
  ...
  ```

Since XOR is its own inverse, encrypting and decrypting are performed using the identical process.

---

## Communication Flow

```text
Alice                            Bob                          Eve
-----                            ---                          ---
Plaintext (P)
   | -- Encrypt using AES-CFB (with key K & IV) -->
   |                Produces Ciphertext (C)
   | <-- Decrypt using same K & IV -- 
Plaintext (P)
```

- **Note:**  
  Eve can observe the ciphertext but cannot decrypt it without knowing the secret key **K**.

---

## Key Parameters

- **Key (K):**  
  AES key of 128, 192, or 256 bits.

- **IV (Initialization Vector):**  
  A random, unique value for each encryption session.

- **Segment Size:**  
  - Typically the block size (e.g., 128 bits for AES).  
  - Can be adjusted to smaller sizes (e.g., 8 bits per segment) for stream-like behavior.

---

## Security Considerations

- **Advantages:**
  - **No Padding Required:**  
    Able to encrypt messages of arbitrary length.
  - **Stream Cipher Behavior:**  
    Suitable for real-time or variable-length data transmission.

- **Risks:**
  - **IV Reuse:**  
    Must never reuse the same (key, IV) pair; doing so reuses the keystream and compromises security.
  - **Error Propagation:**  
    A single bit error in a ciphertext segment corrupts that segment and the next one.
  - **Lack of Authentication:**  
    CFB does not provide message integrity. It is advisable to combine it with an HMAC or use an authenticated encryption mode (such as AES-GCM or ChaCha20-Poly1305).

---

## Usage Context

- **Historical Applications:**  
  - Early secure communication protocols.
  - Systems requiring streaming encryption without padding.

- **Modern Alternatives:**  
  - CTR mode for parallel processing.
  - AEAD modes (e.g., AES-GCM, ChaCha20-Poly1305) for combined confidentiality and integrity.

In [1]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes

# --- Key & IV ---
key = get_random_bytes(16)   # AES-128
iv = get_random_bytes(16)    # 16-byte IV

# --- Encrypt ---
cipher_enc = AES.new(key, AES.MODE_CFB, iv=iv)

plaintext = b"CFB mode: encrypting arbitrary length"
ciphertext = cipher_enc.encrypt(plaintext)

print("Ciphertext (hex):", ciphertext.hex())

# --- Decrypt ---
cipher_dec = AES.new(key, AES.MODE_CFB, iv=iv)
recovered = cipher_dec.decrypt(ciphertext)

print("Recovered:", recovered)


Ciphertext (hex): 37ea60aa50de9991cc68d4093c1bd18d3ea8ca59807c1917bedd7000c9955a938889d33746
Recovered: b'CFB mode: encrypting arbitrary length'
